# Capstone Segmenting and Clustering Neighborhoods in Toronto

## Step 1: Scape Wiki web page and make pd DataFrame

In [1]:
# import library
import pandas as pd
import numpy as np

#### Read table from wikipedia

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
data = df[0]

In [4]:
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# Remove data where Borough is not assigned
data = data[data['Borough']!='Not assigned']

In [6]:
data.shape

(211, 3)

#### Merge the same postcode row to a single record
#### Here we assume the same Postcode always has the same borough

In [7]:

data = data.groupby('Postcode').agg({'Borough':'first', 'Neighbourhood': ','.join}).reset_index()

In [8]:
data

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [9]:
data['Neighbourhood'].loc[data['Neighbourhood']=='Not assigned'] = data['Borough'].loc[data['Neighbourhood']=='Not assigned']

In [10]:
data.shape

(103, 3)

## Step 2. Use Geocoder to add coordinates to the DataFrame

In [11]:
import geocoder # import geocoder

def getLatLng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
    
    return lat_lng_coords
    #latitude = lat_lng_coords[0]
    #longitude = lat_lng_coords[1]

In [12]:
#import geocoder
#postal_code = "M1J"
#ll = getLatLng(postal_code)
for i in range(len(data)):
    postal_code = data.loc[i, 'Postcode']
    lagLng = getLatLng(postal_code)
    data.loc[i, 'Latitude'] = lagLng[0]
    data.loc[i, 'Longitude'] = lagLng[1]

In [13]:
data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## Step 3. Explore and cluster the neighborhoods in Toronto

In [14]:
location = geocoder.arcgis('Toronto, Ontario')
location = location.latlng
latitude = location[0]
longitude = location[1]
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.648690000000045, -79.38543999999996.


In [15]:
import folium
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(data['Latitude'], data['Longitude'], data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
# Only consider borough that contains the word 'Toronto'
torontoData = data[data['Borough'].str.contains('Toronto')]

In [17]:
torontoData.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676531,-79.295425
41,M4K,East Toronto,"The Danforth West,Riverdale",43.683178,-79.355105
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314667
43,M4M,East Toronto,Studio District,43.660629,-79.334855
44,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133


In [18]:
import requests
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
torontoVenues = getNearbyVenues(names=torontoData['Neighbourhood'],
                                   latitudes=torontoData['Latitude'],
                                   longitudes=torontoData['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [21]:
print(torontoVenues.shape)
torontoVenues.head()

(1764, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676531,-79.295425,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676531,-79.295425,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676531,-79.295425,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676531,-79.295425,Domino's Pizza,43.679058,-79.297382,Pizza Place
4,The Beaches,43.676531,-79.295425,Upper Beaches,43.680563,-79.292869,Neighborhood


In [22]:
# one hot encoding
torontoVenuesOnehot = pd.get_dummies(torontoVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torontoVenuesOnehot['Neighborhood'] = torontoVenues['Neighborhood'] 

torontoVenuesOnehot.head()

,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
torontoGrouped = torontoVenuesOnehot.groupby('Neighborhood').mean().reset_index()
torontoGrouped

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.030000,0.000000,0.010000,0.00,0.000000,0.030000,0.000000,0.000000,...,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.015873,0.00,0.000000,0.000000,0.000000,0.015873,...,0.00,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.015152,0.00,0.015152,0.000000,0.000000,0.000000,...,0.00,0.000000,0.030303,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.030000,0.000000,0.010000,0.00,0.010000,0.020000,0.000000,0.000000,...,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.014286,0.000000,0.000000,...,0.00,0.000000,0.014286,0.014286,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.010000,0.000000,0.010000,0.01,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.021978,0.00,0.021978,0.000000,0.000000,0.000000,...,0.00,0.000000,0.043956,0.000000,0.000000,0.043956,0.010989,0.000000,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.100000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.012346,0.012346,0.000000,0.000000,0.00,0.012346,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.012346,0.012346


In [24]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 8

torontoGroupedClustering = torontoGrouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torontoGroupedClustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 0, 4,
       6, 0, 5, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 7], dtype=int32)

In [25]:
torontoGrouped.insert(0, 'Cluster Labels', kmeans.labels_)
torontoGrouped = torontoGrouped[['Cluster Labels', 'Neighborhood']]
torontoGrouped

,Cluster Labels,Neighborhood
0,0,"Adelaide,King,Richmond"
1,0,Berczy Park
2,0,"Brockton,Exhibition Place,Parkdale Village"
3,0,Business Reply Mail Processing Centre 969 Eastern
4,0,"CN Tower,Bathurst Quay,Island airport,Harbourf..."
5,0,"Cabbagetown,St. James Town"
6,0,Central Bay Street
7,0,"Chinatown,Grange Park,Kensington Market"
8,0,Christie
9,0,Church and Wellesley


In [26]:
torontoMerged = torontoGrouped.join(torontoData.set_index('Neighbourhood'), on='Neighborhood')

torontoMerged.head()

,Cluster Labels,Neighborhood,Postcode,Borough,Latitude,Longitude
0,0,"Adelaide,King,Richmond",M5H,Downtown Toronto,43.649700,-79.382582
1,0,Berczy Park,M5E,Downtown Toronto,43.645160,-79.373675
2,0,"Brockton,Exhibition Place,Parkdale Village",M6K,West Toronto,43.639410,-79.424362
3,0,Business Reply Mail Processing Centre 969 Eastern,M7Y,East Toronto,43.648690,-79.385440
4,0,"CN Tower,Bathurst Quay,Island airport,Harbourf...",M5V,Downtown Toronto,43.640815,-79.399538


In [27]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torontoMerged['Latitude'], torontoMerged['Longitude'], torontoMerged['Neighborhood'], torontoMerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters